In [ ]:
colab=False
if colab:
  !git clone https://github.com/cthiounn/images-generated-by-dalle-mini.git

In [ ]:
if colab:
  %run "images-generated-by-dalle-mini/util/util_extract_test1kFlickr30k.ipynb"

In [ ]:
%store -r df

In [ ]:
!pip install -q transformers flax==0.3.5
!pip install -q git+https://github.com/patil-suraj/vqgan-jax.git  # VQGAN model in JAX
!pip install -q git+https://github.com/borisdayma/dalle-mini.git@26651ddd777f73dd18dade8bf543a6a8732d3669  # Model files

In [ ]:
from dalle_mini.model import CustomFlaxBartForConditionalGeneration
from transformers import BartTokenizer
import jax
import random
from tqdm.notebook import tqdm, trange
from vqgan_jax.modeling_flax_vqgan import VQModel
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, FlaxCLIPModel



# make sure we use compatible versions
DALLE_REPO = "flax-community/dalle-mini"
DALLE_COMMIT_ID = "4d34126d0df8bc4a692ae933e3b902a1fa8b6114"
# make sure we use compatible versions
VQGAN_REPO = "flax-community/vqgan_f16_16384"
VQGAN_COMMIT_ID = "90cc46addd2dd8f5be21586a9a23e1b95aa506a9"
# set up model and processor
clip = FlaxCLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# set up tokenizer and model
tokenizer = BartTokenizer.from_pretrained(DALLE_REPO, revision=DALLE_COMMIT_ID)
model = CustomFlaxBartForConditionalGeneration.from_pretrained(
    DALLE_REPO, revision=DALLE_COMMIT_ID
)
vqgan = VQModel.from_pretrained(VQGAN_REPO, revision=VQGAN_COMMIT_ID)



In [ ]:
import os
import re
m= lambda s: re.findall(r'\d+',s)
root="./images-generated-by-dalle-mini/" if colab else "./" 
flickr30k_id_already_done=(set([ m(i)[0] for i in os.listdir(root+"images/flickr30k")]))


In [ ]:
for index, (imgid,first_sentence,filename) in df.iterrows():

    index=int(index)
    if str(index) not in flickr30k_id_already_done and index>500:
        # set a prompt
        prompt = first_sentence.strip()

        # tokenize the prompt
        tokenized_prompt = tokenizer(
            prompt, return_tensors="jax", padding="max_length", truncation=True, max_length=128
        )

        n_predictions = 3

        # create random keys
        seed = random.randint(0, 2 ** 32 - 1)
        key = jax.random.PRNGKey(seed)
        subkeys = jax.random.split(key, num=n_predictions)


        encoded_images = [
            model.generate(**tokenized_prompt, do_sample=True, num_beams=1, prng_key=subkey)
            for subkey in tqdm(subkeys)
        ]

        encoded_images = [img.sequences[..., 1:] for img in encoded_images]

        decoded_images = [
            vqgan.decode_code(encoded_image) for encoded_image in tqdm(encoded_images)
        ]

        clipped_images = [img.squeeze().clip(0.0, 1.0) for img in decoded_images]


        # convert to image
        images = [
            Image.fromarray(np.asarray(img * 255, dtype=np.uint8)) for img in clipped_images
        ]
        for i,j in enumerate(images):
            j.save(root+"images/flickr30k/"+str(index)+"_"+str(imgid)+"_"+str(i)+".jpeg")
    else:
        print("flickr30k_id_already_done:",index)
        # evaluate scores
#         inputs = processor(text=prompt, images=images, return_tensors="np")
#         logits = clip(**inputs).logits_per_image
#         scores = jax.nn.softmax(logits, axis=0).squeeze()  # normalize and sum all scores to 1

#         # rank images by score
#         print(f"Prompt: {prompt}\n")
#         for idx in scores.argsort()[::-1]:
#             print(f"Score: {scores[idx]}")
#             display(images[idx])
#             print()

In [ ]:
# !nvidia-smi
# !nvcc --version
# import torch
# torch.cuda.is_available()
# # Output would be True if Pytorch is using GPU otherwise it would be False.
# import tensorflow as tf
# tf.test.gpu_device_name()
# # Standard output is '/device:GPU:0'
# !cat /usr/local/lib/python3.6/dist-packages/external/local_config_cuda/cuda/cuda/cuda_config.h | grep TF_CUDA_VERSION
# jax.devices()